<font face="Calibri" size="2"> <i>Open SAR Toolkit - Tutorial 4a, version 1.1, Nobember 2019. Andreas Vollrath, ESA/ESRIN phi-lab</i>
</font>

![title](auxiliary/header_image.PNG)

--------

# OST Tutorial IV - A
## How to do a batch processing of Sentinel-1 GRD data subset

--------

**Short description**

This notebook shows f OST for the batch processing of GRD data using the *Sentinel1_GRDBatch* class. This is a subclass of the *Sentinel1* class, and thus inherits all the functionalities of the *Generic* and the *Sentinel1* classes presented in Tutorial II. In addition, functions for the generation of calibrated backscatter products are provided. Within the example a subset area will be produced, thus avoiding the overload of processing the full images.

--------

**Requirements**

- a PC/Mac with at least 16GB of RAM
- about 15GB of free disk space
- a Copernicus Open Data Hub user account, valid for at least 7 days (https://scihub.copernicus.eu)
--------

**NOTE:** all cells that have an * after its number can be executed without changing any code. 

### 1* - Import of Libraries

In this step we import some standard python libraries for OS independent path handling as well as the *Sentinel1_GRDBatch* class thta handles the full workflow from search, download and processing of multiple GRD scenes. In addition, the OST helper module *vector* is loaded to create an AOI based on Point coordinates, and the *raster* module for creating a time-series animation.

In [ ]:
import os
from os.path import join
from pathlib import Path
from pprint import pprint

from ost import Sentinel1_GRDBatch
from ost.helpers import vector, raster

### 2* - Set up the project 

Here you going to initialize the *Sentinel1_GRDBatch* class by determining the project folder, the AOI and the start and end date. Since you should be already familiar with the *search* and *refine* functions, we execute them within the same cell.

In [ ]:
# define a project directory
home = str(Path.home())
# create a processing directory
project_dir = join(home, 'OpenSarToolkit', 'Tutorial_4a')

# define aoi with a 2 point coordinates and create a buffer
lat, lon = '47.25', '14'
aoi = vector.latlon_to_wkt(lat, lon, buffer_meter=10000, envelope=True)

#define the start and end date
start = '2019-05-01'
end = '2019-05-31'

# initialize the class to s1_grd instance
s1_grd = Sentinel1_GRDBatch(
    project_dir=project_dir,
    aoi = aoi,
    start = start,
    end = end
)

# trigger the search
s1_grd.search()

# optional: once you did the search the first time, you can load 
# the full inventory uncommenting the follwoing 2 lines
# and commenting the search command
#s1_grd.inventory_file = join(s1_grd.inventory_dir, 'full.inventory.shp')
#s1_grd.read_inventory()

# do the refinement
s1_grd.refine()

### 3* - Plot refined data inventory

Here you will visualize the resultant dataframes from the refined search inventory based on the product key.

In [ ]:
#---------------------------------------------------
# for plotting purposes we use this iPython magic
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (19, 19)
#---------------------------------------------------

# search command
key = 'DESCENDING_VVVH'
# we plot the full Inventory on a map
s1_grd.plot_inventory(s1_grd.refined_inventory_dict[key], transparency=.1)

### 4* - Download of GRD scenes

As already shown in Tutorial II, you will download the scenes based on the refined inventory dataframe for the respective produckt key.

In [ ]:
s1_grd.download(s1_grd.refined_inventory_dict[key])

### 5* - Set ARD parameters

Similar to the *Sentinel1-Scene* class (Tutorial I and III), the *Sentinel1-GRDBatch* class handles the defintion of ARD types in a hierarchical dictionary structure. You can use the same types and steps to customize as for the *Sentinel1-Scene* class.

In [ ]:
# single scene ARD parameters
s1_grd.ard_parameters['single ARD']['resolution'] = 50
s1_grd.ard_parameters['single ARD']['product type'] = 'GTCgamma'
s1_grd.ard_parameters['single ARD']['create ls mask'] = False

# time-series ARD
s1_grd.ard_parameters['time-series ARD']['remove mt speckle'] = False

# in case we want to use an external dem
#s1_grd.set_external_dem('path/to/externaldem')

pprint(s1_grd.ard_parameters)

### 6* - Run the batch routine

To process all the data, including time-series and timescans is as easy as one command. All the complexity is handled behind, and you just have to wait, since processing can take quite a while.

**Note** that as a last argument we submit the AOI (given as WKT) for subsetting.

In [ ]:
s1_grd.grds_to_ard(
    inventory_df=s1_grd.refined_inventory_dict[key], 
    timeseries=True, 
    timescan=True, 
    mosaic=False, 
    overwrite=False, 
    subset=s1_grd.aoi
)

### 7* - Create a time-series animation

For interactive presentations it is nice to have animated "movies". The following command allows you to create animated time-series of oyur processed data.

In [ ]:
# define Time-series folder
ts_folder = join(s1_grd.processing_dir, '95', 'Timeseries')

# create the animation
raster.create_timeseries_animation(
    timeseries_folder=ts_folder, 
    product_list=['bs.VV', 'bs.VH'], 
    out_folder=s1_grd.processing_dir,
    shrink_factor=5, 
    add_dates=True
)